In [1]:
%run ../utils.py

In [2]:
import pandas as pd
import numpy as np
import gc, sys
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## build matrix data : recipes x ingredients
1. open PP_recipes.csv
2. create a set that enumerates all possible ingredients
3. build the matrix with dim : nbOfRecipes x nbOfPossibleIngredients

In [3]:
#read the recipes file
pp_recipes = pd.read_csv('../../data/PP_recipes.csv')
pp_recipes.sort_values(['i'], inplace=True)
pp_recipes.head(3)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
46053,40893,0,"[40480, 1454, 16201, 2056, 955, 541, 11332, 82...","[[1424, 8876, 11007], [3484, 21453], [38966, 2...","[40480, 40482, 23667, 11007, 240, 21453, 240, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",0,"[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500..."
108526,44394,1,"[40480, 34712, 22683, 11274, 5409, 29868, 40481]","[[5343, 535, 2044, 5409, 7087], [17869, 6020],...","[40480, 40482, 12172, 1281, 5409, 7087, 240, 6...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[912, 7557, 2499, 5382]"
41248,85009,2,"[40480, 12187, 13995, 571, 14719, 40481]","[[1061, 494, 813, 2141], [31843], [30645, 4785...","[40480, 40482, 2572, 19472, 31757, 512, 823, 4...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",2,"[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169..."


In [4]:
# we need to create a collection of ingredients_ids to build vectors of recipes
set_ingredients = set()

for i in tqdm(range(len(pp_recipes['ingredient_ids']))):
    current = [int(x.strip()) for x in pp_recipes['ingredient_ids'].values[i][1:-1].split(',')]
    set_ingredients.update(current)
    
print('Number of ingredients in total :', len(set_ingredients))


Number of ingredients in total : 7993


In [5]:
list_ingredients = list(set_ingredients)

#initialize the matrix
np_matrix = np.zeros((len(pp_recipes), list_ingredients[-1] + 1),dtype=np.uint8)

#populate it
for i in tqdm(range(len(pp_recipes['ingredient_ids']))):
    for x in pp_recipes['ingredient_ids'].values[i][1:-1].split(','):
        np_matrix[i,int(x.strip())] = 1


In [6]:
#wrap the matrix in a pandas dataframe
pd_matrix = pd.DataFrame(np_matrix,columns=range(list_ingredients[-1] + 1), index=pp_recipes['i'].values,dtype=np.uint8)

del np_matrix
gc.collect()

pd_matrix.head(5)

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#check matrix consistensy with user ID 23
assert(pd_matrix.loc[23,389]==1)
assert(pd_matrix.loc[23,7655]==1)
assert(pd_matrix.loc[23,6270]==1)
assert(pd_matrix.loc[23,1527]==1)
assert(pd_matrix.loc[23,3406]==1)

## Apply SVD decomposition for dimensionality reduction
As the matrix is sparse : we have about 8000 ingredients but recipes contains on average 10 ingredients
we use the scipy.sparse.linalg.svds for the SVD decomposition

In [8]:
from scipy.sparse.linalg import svds

#takes around 20 minutes
U, S, V = svds(pd_matrix.values.astype(np.float32),k=2000)

In [9]:
reduced_matrix = U * S
print('shape of reduced matrix :', reduced_matrix.shape)

shape of reduced matrix : (178265, 2000)


## Apply PCA dimensionality reduction

In [ ]:
C = pd_matrix - np.mean(pd_matrix, axis = 0)
Cov = C @ C.T

from scipy.sparse.linalg import eigsh
eigvect, eigval = eigsh(Cov, k=1000)

# distance Measure between recipes : Cosine Similarity
1. We will compute the cosineSimilarity for recipe id #7 on the matrix and reduced_matrix (SVD)
2. We will display top 5 recommandations (the 5 recipes that are the closest)

In [10]:
def OneVSAllCosineSimilarity(oneId, featuresMatrix):
    """For a matrix of features, computes the cosine similarity for one index *oneId* against all ids 
       including *oneId*
        oneId :  
            int, the id on wich the cosine similarity must be compute
        featuresMatrix: 
            numpy array, matrix of features : items x features
    :Returns:
         A array of cosine similarity of size (1, items-1)
    :Example:
        cosineSimilarity = OneVSAllCosineSimilarity(0, pd_matrix)
    """
    oneVector = featuresMatrix[oneId,:].reshape(1, -1)
    
    return cosine_similarity(oneVector, featuresMatrix)[0]

In [11]:
#execute cosine similarity on user #7
result = OneVSAllCosineSimilarity(7, pd_matrix.values)
result_reduced = OneVSAllCosineSimilarity(7, reduced_matrix)

In [12]:
pp_recipes = pd.read_csv('../../data/PP_recipes.csv')
raw_recipes = pd.read_csv('../../data/RAW_recipes.csv')

In [13]:
def GetClosestRecipes(oneId, cosine_vector, N, verbose=True):
    """Return the topN recipes in cosine_vector considering oneId in cosine_vector
        oneId :  
            int, the id on wich the cosine similarity has been compute
        cosine_vector: 
            numpy array, array of cosine vector for all recipes
        N:
            integer, number of top N recipes to return
        verbose:
            Boolean, if true, print topN recipes info
    :Returns:
         two arrays. A array of topN recipes ID (i) and a array of topN scores
    :Example:
        GetClosestRecipes(0, result, 20)
    """
    locationOneId = np.where(np.argsort(cosine_vector)[::-1] == oneId)[0][0]
    topNrecipes = None
    
    if locationOneId >= N:
        topNrecipes = np.argsort(cosine_vector)[::-1][:N]
        topNScore = np.sort(cosine_vector)[::-1][:N]
    else:
        topNrecipes = np.argsort(cosine_vector)[:(-N-2):-1][np.arange(N+1)!=locationOneId]
        topNScore = np.sort(cosine_vector)[:(-N-2):-1][np.arange(N+1)!=locationOneId]
    
    if verbose:
        print(f'The recipe you are looking for ({oneId}) is:')
        display(GetRecipesInformationsbyId([oneId], pp_recipes, raw_recipes))
        print(f'the top {N} closest recipes are:')
        topN = GetRecipesInformationsbyId(topNrecipes, pp_recipes, raw_recipes)
        topN['score']=topNScore
        topN.sort_values('score', inplace=True, ascending=False)
        display(topN)
    
    gc.collect()
    
    return topNrecipes, topNScore

GetClosestRecipes(7, result, 5, True)
GetClosestRecipes(7, result_reduced, 5, True)

The recipe you are looking for (7) is:


,name,ingredients
165178,potato crab chowder,"['butter', 'onion', 'garlic', 'potatoes', 'flo..."


the top 5 closest recipes are:


,name,ingredients,score
38091,celery vichyssoise,"['onion', 'butter', 'flour', 'chicken broth', ...",0.647150
115668,judy s clam chowder,"['minced clams', 'onion', 'potatoes', 'bacon',...",0.640513
146917,old fashioned chicken casserole,"['flour', 'salt and pepper', 'butter', 'chicke...",0.588348
158154,peppery scalloped potatoes,"['cream of mushroom soup', 'milk', 'salt', 'ca...",0.588348
223068,very easy cheesy scalloped potatoes,"['potatoes', 'onions', 'butter', 'flour', 'sal...",0.588348


The recipe you are looking for (7) is:


,name,ingredients
165178,potato crab chowder,"['butter', 'onion', 'garlic', 'potatoes', 'flo..."


the top 5 closest recipes are:


,name,ingredients,score
33527,cajun corn chowder,"['butter', 'onion', 'flour', 'chicken broth', ...",0.647281
115668,judy s clam chowder,"['minced clams', 'onion', 'potatoes', 'bacon',...",0.640638
146917,old fashioned chicken casserole,"['flour', 'salt and pepper', 'butter', 'chicke...",0.590066
158154,peppery scalloped potatoes,"['cream of mushroom soup', 'milk', 'salt', 'ca...",0.588944
223068,very easy cheesy scalloped potatoes,"['potatoes', 'onions', 'butter', 'flour', 'sal...",0.588474


(array([114014,  95251, 176883, 141197, 154171]),
 array([0.6472813, 0.6406385, 0.5900662, 0.5889437, 0.5884744],
       dtype=float32))

In [14]:
print(cosine_similarity(pd_matrix.loc[7,:].values.reshape(1, -1), pd_matrix.loc[152668,:].values.reshape(1, -1)))
print(np.argwhere(pd_matrix.loc[166329,:].values.reshape(1, -1)==1))
print(np.argwhere(pd_matrix.loc[0,:].values.reshape(1, -1)==1))

[[0.58834841]]
[[   0 1093]
 [   0 3217]
 [   0 3502]
 [   0 5006]
 [   0 6832]]
[[   0 1257]
 [   0 2127]
 [   0 2778]
 [   0 3217]
 [   0 3384]
 [   0 3502]
 [   0 5006]
 [   0 6654]
 [   0 7979]]


# implement a global method for recommandation
1. loop over all recipes rated by a user to identify the 5 closest one (for each recipes)
2. provide the 'aggregate' 5 closest ones for all recipes (room to improve heuristic)

In [15]:
#read the train file to get user's data
interactions_train = pd.read_csv('../../data/generated/interactions_train.csv')

def getCBFRecommandation(userId, topN, listRecipesId, ing_matrix):
    """Return the topN recipes in cosine_vector considering oneId in cosine_vector
        oneId :  
            int, the id on wich the cosine similarity has been compute
        cosine_vector: 
            numpy array, array of cosine vector for all recipes
        N:
            integer, number of top N recipes to return
        verbose:
            Boolean, if true, print topN recipes info
    :Returns:
         two arrays. A array of topN recipes ID (i) and a array of topN scores
    :Example:
        GetClosestRecipes(0, result, 20)
    """

    dict_recipes = dict()

    for i in tqdm(listRecipesId):
        similarity = OneVSAllCosineSimilarity(i, ing_matrix)
        recipes, scores = GetClosestRecipes(i, similarity, topN, True)
        for recipe, score in zip(recipes, scores):
            if recipe not in interactions_train[interactions_train['u']==userId]['i'].values:
                if (recipe not in dict_recipes) or (dict_recipes[recipe] < score):
                    dict_recipes[recipe] = score

    #exclude recipes already performed by user
    #todo

    dict_recipes = sorted(dict_recipes.items(), key=lambda k: k[1], reverse=True)

    return dict_recipes[:topN]


reco = getCBFRecommandation(16, 5, interactions_train[interactions_train['u']==16]['i'].values[:3],pd_matrix.values)
reco

The recipe you are looking for (99787) is:


,name,ingredients
213826,to die for crock pot roast,"['beef roast', 'brown gravy mix', 'dried itali..."


the top 5 closest recipes are:


,name,ingredients,score
1289,3 pack pot roast,"['beef roast', 'brown gravy mix', 'italian sal...",1.000000
5748,another crock pot roast,"['beef roast', 'ranch dressing mix', 'liquid s...",1.000000
76000,easy company pot roast,"['beef roast', 'brown gravy mix', 'dried itali...",0.894427
112293,italian beef au jus,"['boneless beef roast', 'au jus mix', 'italian...",0.816497
213219,three packs and a roast,"['beef roast', 'ranch dressing mix', 'italian ...",0.670820


The recipe you are looking for (75726) is:


...

,name,ingredients,score
2261,abm dinner rolls,"['water', 'eggs', 'butter', 'sugar', 'salt', '...",0.777778
67538,crusty cheddar bread,"['active dry yeast', 'water', 'cottage cheese'...",0.755929
141908,my hard rolls for bratwurst,"['flour', 'water', 'salt', 'instant yeast', 's...",0.755929
151276,overnight butter rolls,"['dry yeast', 'water', 'eggs', 'flour', 'sugar...",0.755929
182773,scoop and bake dinner rolls,"['all-purpose flour', 'sugar', 'salt', 'fast r...",0.755929


The recipe you are looking for (79338) is:


,name,ingredients
132340,mean chef s chicken with marsala mushrooms an...,"['boneless skinless chicken breast', 'kosher s..."


the top 5 closest recipes are:


,name,ingredients,score
18575,basic vinaigrette with variations,"['garlic cloves', 'dijon mustard', 'white wine...",0.559017
98665,grilled chicken with radish salad,"['boneless chicken breasts', 'olive oil', 'lim...",0.527046
112639,italian grilled mushrooms,"['cremini mushroom', 'salt', 'pepper', 'garlic...",0.527046
188159,skillet rosemary chicken,"['red potatoes', 'kosher salt', 'fresh rosemar...",0.516398
218787,tuscan potatoes mint and garlic roasted potatoes,"['small red potato', 'garlic cloves', 'olive o...",0.516398


In [17]:
reco = getCBFRecommandation(16, 5, interactions_train[interactions_train['u']==16]['i'].values[:3],reduced_matrix)
reco

The recipe you are looking for (99787) is:


,name,ingredients
213826,to die for crock pot roast,"['beef roast', 'brown gravy mix', 'dried itali..."


the top 5 closest recipes are:


,name,ingredients,score
1289,3 pack pot roast,"['beef roast', 'brown gravy mix', 'italian sal...",1.000000
5748,another crock pot roast,"['beef roast', 'ranch dressing mix', 'liquid s...",1.000000
76000,easy company pot roast,"['beef roast', 'brown gravy mix', 'dried itali...",0.894377
112293,italian beef au jus,"['boneless beef roast', 'au jus mix', 'italian...",0.816292
213219,three packs and a roast,"['beef roast', 'ranch dressing mix', 'italian ...",0.750165


The recipe you are looking for (75726) is:


...

,name,ingredients,score
2261,abm dinner rolls,"['water', 'eggs', 'butter', 'sugar', 'salt', '...",0.777826
67538,crusty cheddar bread,"['active dry yeast', 'water', 'cottage cheese'...",0.756062
72355,dinner rolls bread machine,"['egg', 'water', 'bread flour', 'butter', 'sal...",0.756062
106007,holiday dinner rolls bread machine,"['water', 'eggs', 'butter', 'salt', 'bread flo...",0.756062
169614,quincy s yeast rolls,"['water', 'butter', 'egg', 'bread flour', 'sug...",0.756062


The recipe you are looking for (79338) is:


,name,ingredients
132340,mean chef s chicken with marsala mushrooms an...,"['boneless skinless chicken breast', 'kosher s..."


the top 5 closest recipes are:


,name,ingredients,score
46530,chicken with pearl onion and cremini mushroom ...,"['boneless skinless chicken breasts', 'salt', ...",0.589481
98665,grilled chicken with radish salad,"['boneless chicken breasts', 'olive oil', 'lim...",0.575796
112639,italian grilled mushrooms,"['cremini mushroom', 'salt', 'pepper', 'garlic...",0.555284
181618,sauteed swiss chard,"['red swiss chard', 'lite olive oil', 'garlic ...",0.555169
188159,skillet rosemary chicken,"['red potatoes', 'kosher salt', 'fresh rosemar...",0.554538


In [20]:
reco = getCBFRecommandation(16, 5, interactions_train[interactions_train['u']==16]['i'].values,reduced_matrix)
reco

The recipe you are looking for (99787) is:


,name,ingredients
213826,to die for crock pot roast,"['beef roast', 'brown gravy mix', 'dried itali..."


the top 5 closest recipes are:


,name,ingredients,score
1289,3 pack pot roast,"['beef roast', 'brown gravy mix', 'italian sal...",1.000000
5748,another crock pot roast,"['beef roast', 'ranch dressing mix', 'liquid s...",1.000000
76000,easy company pot roast,"['beef roast', 'brown gravy mix', 'dried itali...",0.894377
112293,italian beef au jus,"['boneless beef roast', 'au jus mix', 'italian...",0.816292
213219,three packs and a roast,"['beef roast', 'ranch dressing mix', 'italian ...",0.750165


The recipe you are looking for (75726) is:


...

The recipe you are looking for (1755) is:


,name,ingredients
59201,country sausage gravy,"['pork sausage', 'butter', 'all-purpose flour'..."


the top 5 closest recipes are:


,name,ingredients,score
77931,easy sausage gravy,"['bulk pork sausage', 'butter', 'all-purpose f...",0.816515
87239,fresh sausage waffles,"['pork sausage', 'eggs', 'milk', 'butter', 'al...",0.771670
142457,nacho cheese sauce,"['butter', 'all-purpose flour', 'milk', 'ameri...",0.771523
187128,simple pie crust,"['butter', 'all-purpose flour', 'salt', 'milk']",0.730363
226582,white sauce,"['milk', 'butter', 'all-purpose flour', 'salt'...",0.730325


[(129411, 1.0),
 (135043, 0.9999999),
 (53731, 0.9999999),
 (163459, 0.9999999),
 (173606, 0.9899642)]

In [19]:
#show object in memory
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir()], key=lambda x: x[1], reverse=True)

[('pd_matrix', 1436889127),
 ('U', 1426120112),
 ('reduced_matrix', 1426120112),
 ('raw_recipes', 388410531),
 ('pp_recipes', 254454202),
 ('interactions_train', 6750472),
 ('set_ingredients', 524520),
 ('list_ingredients', 72056),
 ('_6', 40187),
 ('S', 8096),
 ('_3', 3253),
 ('_i13', 1651),
 ('_i15', 1552),
 ('_i10', 685),
 ('_i4', 411),
 ('_i5', 375),
 ('_i14', 295),
 ('In', 272),
 ('_ih', 272),
 ('_i7', 259),
 ('_i6', 255),
 ('Out', 248),
 ('_oh', 248),
 ('_i2', 228),
 ('_i11', 202),
 ('current', 200),
 ('_i3', 188),
 ('_i19', 167),
 ('_i17', 163),
 ('_ii', 163),
 ('_i12', 158),
 ('_i8', 145),
 ('GetClosestRecipes', 144),
 ('GetRecipesInformationsbyId', 144),
 ('OneVSAllCosineSimilarity', 144),
 ('__nonzero__', 144),
 ('cosine_similarity', 144),
 ('getCBFRecommandation', 144),
 ('svds', 144),
 ('tqdm', 144),
 ('_i9', 128),
 ('V', 112),
 ('_', 112),
 ('_16', 112),
 ('_18', 112),
 ('__', 112),
 ('reco', 112),
 ('result', 96),
 ('result_reduced', 96),
 ('__doc__', 89),
 ('__builtin__'